In [1]:
!pip install -q openai "httpx<0.28"

import os
from google.colab import userdata


TYPHOON_API_KEY = userdata.get('TYPHOON_API_KEY')
MODEL = "typhoon-v1.5x-70b-instruct"

In [2]:
from openai import OpenAI


client = OpenAI(
   api_key=TYPHOON_API_KEY,
   base_url='https://api.opentyphoon.ai/v1'
)

chat_completion = client.chat.completions.create(
    model="typhoon-v1.5x-70b-instruct",
    messages=[
        {"role": "user", "content": "สวัสดี"}
    ]
)
print(chat_completion.choices[0].message.content)

สวัสดีค่ะ! ยินดีที่ได้พบคุณ. คุณต้องการความช่วยเหลือเกี่ยวกับอะไรคะ?


In [ ]:
from typing_extensions import TypedDict
from IPython.display import Markdown


class ChatTurn(TypedDict):
    role: str
    content: str

def get_completion(
    prompt: str,
    model: str = MODEL,
    stream: bool = False,
    initial_message: list[ChatTurn]| None = None,
) -> str:
    if initial_message is None:
        initial_message = []
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model, messages=messages, temperature=0, stream=stream
    )
    if not stream:
        return response.choices[0].message.content
    else:
        result = ""
        for chunk in response:
            content = chunk.choices[0].delta.content
            print(content, end="")
            if isinstance(content, str):
                result += content
        return result


prompt = "Explain the concept of prompt engineering."
Markdown(get_completion(prompt, stream=True))

# Chain of Thought prompt

In [ ]:
def cot_prompt(question):
    return f"""
    Question: {question}
    Let's approach this step-by-step:
    1) First, let's understand what the question is asking.
    2) Next, let's identify the key information provided.
    3) Then, let's think about how to solve this problem.
    4) Finally, let's calculate the answer.

    Now, let's go through each step:
    """

question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"
cot_response = get_completion(cot_prompt(question))
print(cot_response)

# Gemini

In [ ]:
import google.generativeai as genai
import os


GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-1.0-pro")
response = model.generate_content(get_completion(cot_prompt(question)))
print(response.text)